<a href="https://colab.research.google.com/github/MyTamila75/DeFi_PanCakeSwapBot/blob/master/flux_1_kontext_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

# Re-instantiate nodes (assuming the previous cells for cloning and installing have been run)
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]() # We need VAEEncode for image-to-image
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]() # We need LoadImage to load our source image

# Define the closestNumber helper function (from your previous code)
def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

with torch.inference_mode():
    # Ensure models are loaded (re-run this block if your Colab session restarted)
    # These lines assume the model files were downloaded to the correct paths in previous cells
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

    # --- IMAGE EDITING PARAMETERS ---
    # Path to your input image. Make sure this image exists in your Colab environment
    # or in your mounted Google Drive (e.g., "/content/drive/MyDrive/my_image.png")
    input_image_path = "/content/test_image.png" # <--- IMPORTANT: Change this to your image path!

    # Create a placeholder image if it doesn't exist for demonstration
    # In a real scenario, you'd upload your own image.
    try:
        Image.open(input_image_path)
    except FileNotFoundError:
        print(f"Input image not found at {input_image_path}. Creating a placeholder.")
        dummy_image = Image.new('RGB', (1024, 1024), color = 'red')
        dummy_image.save(input_image_path)
        print(f"Placeholder image saved to {input_image_path}")

    # The text instruction for editing the image
    positive_prompt = "Change the red background to a lush green forest, add a small, cute brown dog in the bottom left corner, keep the main subject in the center."
    # You can add a negative prompt to guide what you DON'T want
    negative_prompt = "blurry, distorted, ugly, bad anatomy, deformed, extra limbs, watermark, text, signature"

    # Denoise strength: 1.0 means completely ignore the original image (text-to-image from noise)
    # A value between 0.5 and 0.8 is typical for image editing, preserving some of the original structure.
    # Lower values (e.g., 0.2-0.4) for subtle changes, higher values (e.g., 0.6-0.8) for more significant transformations.
    denoise_strength = 0.7 # <--- Adjust this value to control how much the image changes!

    # Other parameters (can be adjusted)
    seed = 0 # Set to 0 for random seed, or a specific number for reproducibility
    steps = 25 # More steps can improve quality, but take longer
    sampler_name = "euler"
    scheduler = "simple"

    # Generate a random seed if set to 0
    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(f"Using seed: {seed}")

    # --- Image-to-Image Workflow ---
    # 1. Load the input image
    # LoadImage.load_image returns a tuple (image_tensor, image_width, image_height, image_channels, image_mode)
    # We only need the image_tensor for VAE encoding.
    input_image_tensor = LoadImage.load_image(input_image_path)[0]

    # 2. Encode the input image into latent space
    # The VAEEncode node expects a pixel tensor (input_image_tensor) and the VAE model.
    # It returns a latent representation.
    initial_latent = VAEEncode.encode(vae, input_image_tensor)[0]

    # 3. Prepare conditioning (positive and negative prompts)
    cond_pos, pooled_pos = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond_pos = [[cond_pos, {"pooled_output": pooled_pos}]]

    cond_neg, pooled_neg = clip.encode_from_tokens(clip.tokenize(negative_prompt), return_pooled=True)
    cond_neg = [[cond_neg, {"pooled_output": pooled_neg}]]

    # 4. Generate noise for the sampler
    noise = RandomNoise.get_noise(seed)[0]

    # 5. Prepare guider, sampler, and sigmas
    guider = BasicGuider.get_guider(unet, cond_pos)[0] # Guider uses positive conditioning
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    # Sigmas for image-to-image start from a higher noise level (denoise_strength)
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, denoise_strength)[0]

    # 6. Run the sampler with the initial_latent (encoded input image)
    # The SamplerCustomAdvanced takes the noise, guider, sampler, sigmas, and the initial_latent
    # It will denoise the initial_latent towards the prompt, guided by the noise schedule.
    sample, sample_denoised = SamplerCustomAdvanced.sample(
        noise, guider, sampler, sigmas, initial_latent,
        negative_cond=cond_neg # Pass negative conditioning here
    )

    # 7. Clear memory and decode the latent image back to pixels
    model_management.soft_empty_cache()
    decoded_image_tensor = VAEDecode.decode(vae, sample)[0].detach()

    # 8. Convert to PIL Image and save
    # ComfyUI's decoded images are typically in the range [0, 1] and in (batch, height, width, channels) format
    # PIL expects (height, width, channels) and values 0-255 (uint8)
    output_image_array = np.array(decoded_image_tensor * 255, dtype=np.uint8)[0]
    output_pil_image = Image.fromarray(output_image_array)

    # Save the output image
    output_image_path = "/content/edited_flux_image.png"
    output_pil_image.save(output_image_path)
    print(f"Edited image saved to: {output_image_path}")

# Display the generated image in the Colab output
output_pil_image


In [ ]:
with torch.inference_mode():
    positive_prompt = "black forest toast spelling out the words 'FLUX DEV', tasty, food photography, dynamic shot"
    width = 1024
    height = 1024
    seed = 0
    steps = 20
    sampler_name = "euler"
    scheduler = "simple"

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])